
<div style="text-align: center;">
    <h1>ReConPatch: anomaly detection</h1>
    <h3>Authors:</h3>
    <p>Dario Loi 1940849, Elena Muia 1938610, Martina Doku 1938629</p>

</div>


<div>
    <h2>0 - Introduction</h2>
    <p>This project aims to reimplement and potentially advance the ReConPatch method proposed in the paper titled  <a href="https://arxiv.org/pdf/2305.16713v3">”ReConPatch:
Anomaly Detection by Linear Modulation of Pretrained Features.”</a> This method addresses the challenge of
anomaly detection by constructing discriminative features through a linear modulation of patch features extracted
from pre-trained models and employs contrastive representation learning to collect and distribute features in a way
that produces a target-oriented and easily separable representation of the data.</p>
</div>


In [ ]:
pip install ema-pytorch

In [ ]:
pip install lightning

In [ ]:
pip install adamp

In [ ]:
import os
from typing import Optional
from lightning import LightningDataModule, LightningModule
import lightning.pytorch as pl
import torch.optim as optim
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
from tqdm import tqdm
from ema_pytorch import EMA
import wandb
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.optim.lr_scheduler import CosineAnnealingLR
from adamp import AdamP
import lightning.pytorch as torchpl
from torchvision.transforms import transforms
from sklearn import random_projection
import numpy as np

<div>
    <h2>1.1 - Data</h2>
    <p>In this study, we used the <a href="https://arxiv.org/pdf/2305.16713v3">MVTec AD</a> dataset
and <a href="https://arxiv.org/pdf/2305.16713v3">BTAD</a> dataset for our experiments</p>
</div>


In [ ]:
#data
#downloaded from https://www.kaggle.com/uciml/pima-indians-diabetes-database
#!wget https://www.mvtec.com/company/research/datasets/mvtec-ad/downloads/mvtec_anomaly_detection.tar.xz


<div>
    <h2>1.2 - Data preprocessing</h2>
    <p>We create a unique data module to feed to the lightning module for both the datasets. The preprocessing will follow the ones specified in the sections 4.3 and 4.4 of the aforementioned paper.</p>
</div>


In [ ]:

class MVTecDataModule(LightningDataModule):
    def __init__(
        self,
        data_dir: str,
        batch_size: int = 16,
        num_workers: int = 4,
        train_val_split: float = 0.8,
        shuffle: bool = True,
        pin_memory: bool = True,
        image_size: int = 256,
        normalize: Optional[transforms.Normalize] = None,
        **kwargs,
    ):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_val_split = train_val_split
        self.shuffle = shuffle
        self.pin_memory = pin_memory
        self.image_size = image_size
        self.normalize = normalize

    def setup(self, stage=None):
        # Define transformations
        transform = transforms.Compose([
            transforms.Resize((self.image_size, self.image_size)),
            transforms.ToTensor(),
            transforms.CenterCrop(size=(224, 224)),
            self.normalize if self.normalize else transforms.Lambda(lambda x: x),
        ])
        transform_test = transforms.Compose([
            transforms.Resize((self.image_size, self.image_size)),
            transforms.ToTensor(),
            transforms.CenterCrop(size=(224, 224)),
        ]
        )

        # Load dataset
        tot_num=0
        for subclass in ["bottle","cable","capsule","carpet","grid","hazelnut","leather","metal_nut","pill","screw","tile","toothbrush","transistor","wood","zipper"]:
            dataset = ImageFolder(os.path.join(self.data_dir, subclass,"train"), transform=transform)
            test_dataset = ImageFolder(os.path.join(self.data_dir, subclass,"test"), transform=transform)
            # Split dataset into train and validation sets
            num_train = int(len(dataset) * self.train_val_split)
            num_val = len(dataset) - num_train
            tot_num+=len(dataset)
            train_dataset, val_dataset = torch.utils.data.random_split(
                dataset, [num_train, num_val])
            #print(subclass)
            #print('num_train',num_train)
            #print('num_val',num_val)
            #add the subclass to the dataset
            if subclass == "bottle":
                self.train_dataset = train_dataset
                self.val_dataset = val_dataset
                self.test_dataset = test_dataset
            else:
                self.train_dataset = torch.utils.data.ConcatDataset([self.train_dataset,train_dataset])
                self.val_dataset = torch.utils.data.ConcatDataset([self.val_dataset,val_dataset])
                self.test_dataset = torch.utils.data.ConcatDataset([self.test_dataset,test_dataset])

        #print the number of images in the dataset
        print("number of images in train dataset",len(self.train_dataset))
        print("number of images in val dataset",len(self.val_dataset))
        print("number of images in test dataset",len(self.test_dataset))


    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=self.shuffle,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
        )

    def test_dataloader(self):
#         # Load test dataset without shuffling
#         transform = transforms.Compose([
#             transforms.Resize((self.image_size, self.image_size)),
#             transforms.ToTensor(),
#             transforms.CenterCrop(size=(224, 224)),
#             self.normalize if self.normalize else transforms.Lambda(lambda x: x),
#         ])
#         for subclass in ["bottle","cable","capsule","carpet","grid","hazelnut","leather","metal_nut","pill","screw","tile","toothbrush","transistor","wood","zipper"]:
#             test_dataset = ImageFolder(os.path.join(self.data_dir, subclass,"test"), transform=transform)
#             if subclass == "bottle":
#                 self.test_dataset = test_dataset
#             else:
#                 self.test_dataset = torch.utils.data.ConcatDataset([self.test_dataset,test_dataset])

        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
        )
    def show_examples(self, dataloader, title):
        
        # Get a batch of samples
        images, labels = next(iter(dataloader))
        # Select one sample from the batch
        image = images[0]
        # Convert the tensor to a numpy array for displaying
        image = image.permute(1, 2, 0).numpy()
        plt.imshow(image)
        plt.title(f'{title}')
        plt.show()

    def show_random_images(self, dataloader, title, num_images=10):
        fig, axs = plt.subplots(2, num_images // 2, figsize=(15, 8))
        axs = axs.flatten()

        # Randomly select images from the dataloader
        selected_indices = random.sample(range(len(dataloader.dataset)), num_images)
        
        for i, idx in enumerate(selected_indices):
            image, mask = dataloader.dataset[idx]
            image = image.permute(1, 2, 0).numpy()

            axs[i].imshow(image)
            axs[i].set_title(f'{title} - Image {i+1}')
            axs[i].axis('off')

        plt.tight_layout()
        plt.show()

In [ ]:

import matplotlib.pyplot as plt
# Example usage
data_module = MVTecDataModule(data_dir='/kaggle/input/mvtec-ad')
data_module.setup()
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
test_loader = data_module.test_dataloader()

'''
# Show examples
data_module.show_examples(train_loader, 'Training Example')
data_module.show_examples(val_loader, 'Validation Example')
data_module.show_examples(test_loader, 'Test Example')
'''


In [ ]:
import random
# Show random images from the training dataloader
data_module.show_random_images(train_loader, 'Training Random Images', num_images=8)

# Show random images from the validation dataloader
data_module.show_random_images(val_loader, 'Validation Random Images', num_images=8)

# Show random images from the test dataloader
data_module.show_random_images(test_loader, 'Test Random Images', num_images=8)

In [ ]:
data_dir=os.path.join('/kaggle/input/mvtec-ad')
datamod=MVTecDataModule(data_dir=data_dir,batch_size=4,num_workers=4,train_val_split=0.8,shuffle=True,pin_memory=True,image_size=256,normalize=None)
datamod.setup()

In [ ]:
#Algorithm defined in PatchCore paper
def obtainCoreset(
        memory_bank,
        l = 1000, 
        eps = 0.5,
        ):
    
    #random linear projection in lower dimension (more details in citation 49)
    memory_bank = memory_bank.detach().cpu().numpy()
    transformer = random_projection.SparseRandomProjection(eps=eps)
    memory_bank = torch.tensor(transformer.fit_transform(memory_bank))
    last_val = memory_bank[0]
    coreset_indices = [0]
    min_distances = torch.linalg.norm(memory_bank - last_val, dim= 1, keepdims = True)

    for _ in tqdm(range(l - 1)):
        distances = torch.linalg.norm(memory_bank - last_val, dim = 1, keepdims = True)
        min_distances = torch.minimum(distances, min_distances)
        index = torch.argmax(min_distances).item()       
        last_val = memory_bank[index]
        min_distances[index] = 0
        coreset_indices.append(index)
    return torch.tensor(coreset_indices)
    

In [ ]:
def relaxedContrastiveLoss(w, delta, m): 
    return torch.mean(w * delta**2 + (1 - w) * torch.max(m - delta, torch.tensor(0.0))**2)


In [ ]:
#We set \sigma = 1 based on the cited papers 18 and 19 in ReConPatch
class ContextualSimilarity(nn.Module):
    def __init__(self, k = 5, alpha = 0.5):
        super(ContextualSimilarity, self).__init__()
        self.k = k
        self.alpha = alpha
        
    def forward(self, z):
        distances = torch.cdist(z,z)  
        topk_dist = -torch.topk(-distances, self.k)[0][:,-1]
        filtering = (distances <= topk_dist.unsqueeze(-1)).float()
        similarity = torch.matmul(filtering, filtering.transpose(0, 1)) / torch.sum(filtering, dim=-1, keepdim=True)
        R = filtering * filtering.transpose(0, 1)
        similarity = torch.matmul(similarity, R.transpose(0, 1)) / torch.sum(R, dim=-1, keepdim=True)
        return self.alpha * (similarity + similarity.transpose(0, 1))
        
        
class PairwiseSimilarity(nn.Module):
    def __init__(self, sigma = 1.0):
        super(PairwiseSimilarity, self).__init__()
        self.sigma = sigma
    def forward(self,z):
        return torch.exp(-(torch.cdist(z,z)/self.sigma))


class ReConPatch(LightningModule):
    def __init__(
        self,
        input_dim = 1536,
        emb_dim = 512,
        proj_dim = 1024,
        alpha = 0.5,
        margin=0.1,
        
    ):  
        super(ReConPatch, self).__init__() 
        self.memory_bank = []
        self.memory_bank_sample = []
        
        self.fmap =[]
        
        #self.auroc = AUROC(task="multiclass", num_classes=15)
        self.margin = margin
        self.alpha = alpha

        self.frac_rate= 0.01
        self.eps_coreset = 0.5
        self.wr_model = torch.hub.load('pytorch/vision:v0.13.0', 'wide_resnet50_2', pretrained=True)
        
        def hook(module, input, output):
            self.fmap.append(output)
        self.wr_model.layer2.register_forward_hook(hook)            
        self.wr_model.layer3.register_forward_hook(hook)
        #setup network 1
        self.repr_layer = nn.Linear(input_dim, emb_dim)
        self.proj_layer = nn.Linear(emb_dim, proj_dim)
        
        #setup network 2
        self.repr_layer_2=nn.Linear(input_dim, emb_dim)
        self.proj_layer_2=nn.Linear(emb_dim, proj_dim)
        self.ema_repr = EMA(self.repr_layer_2)
        self.ema_proj = EMA(self.proj_layer_2) 
        #???
        with torch.no_grad():
            self.proj_layer.weight.copy_(torch.randn_like(self.proj_layer.weight))
            self.proj_layer.bias.copy_(torch.randn_like(self.proj_layer.bias))
            self.repr_layer.weight.copy_(torch.randn_like(self.repr_layer.weight))
            self.repr_layer.bias.copy_(torch.randn_like(self.repr_layer.bias))
        self.ema_repr.update()
        self.ema_proj.update()
        
        self.pairwise_sim=PairwiseSimilarity()
        self.contextual_sim=ContextualSimilarity()
    def forward(self, x):
        '''
        x has dimensions B x C x H x W (batch channels height width)
        '''
        #-----------FROM PRETRAINED MODEL TO FEATURE MAP
        self.fmap = []
        self.memory_bank_sample = []
        
        y = self.wr_model(x)
        
        #NOTE: We are taking as dimensions the ones of the feature map with higher resolution as specified in
        #chapter 3.1 of the patchcore paper (https://arxiv.org/pdf/2106.08265)
        dimensions = (
                int(torch.Tensor([t.shape[-2] for t in self.fmap]).max().item()),
                int(torch.Tensor([t.shape[-1] for t in self.fmap]).max().item())
            )
        
        blur = nn.AvgPool2d(3, stride = 1)
        resizer = nn.AdaptiveAvgPool2d(dimensions)
        preprocess = lambda t : resizer(blur(t))
        feature_stacks = torch.cat([preprocess(m) for m in self.fmap], dim=1)
        
        
        #-----------RECONPATCH
        feature_stacks_reshaped = feature_stacks.reshape(feature_stacks.shape[1], -1).T
        #feature_stacks_reshaped_cpu = feature_stacks_reshaped.detach().cpu().numpy()
        self.memory_bank_sample.append(feature_stacks_reshaped)
        
        
        #----------network1 pass
        h1=self.ema_repr(feature_stacks_reshaped)
        z1=self.ema_proj(h1)
        
        p_sim=self.pairwise_sim(z1)
        c_sim=self.contextual_sim(z1)
        
        w=self.alpha*p_sim+(1-self.alpha)*c_sim
        #----------network2 pass
        h2=self.repr_layer_2(feature_stacks_reshaped)
        z2=self.proj_layer_2(h2)
        
        
        pairwise_distances = torch.cdist(z2, z2, p=2)

        # Add a small epsilon to avoid division by zero when taking square root
        epsilon = 1e-9
        distances = torch.sqrt(pairwise_distances + epsilon)
        delta=distances/torch.mean(distances)
        
        return w ,delta
         
    def training_step(self, batch, batch_idx):
        x, y = batch
        w,delta = self(x)
        loss = relaxedContrastiveLoss(w, delta, self.margin)
        
        if self.frac_rate < 1:
            
            self.memory_bank_sample = torch.cat(self.memory_bank_sample, axis = 0)
            if self.memory_bank == []:
                self.memory_bank = self.memory_bank_sample
                mem_concat = self.memory_bank
            else:

                mem_concat = torch.cat([self.memory_bank,self.memory_bank_sample], axis = 0)
            
            num_coreset_points = int(self.frac_rate * mem_concat.shape[0])
            coreset_indices = obtainCoreset(mem_concat, l = num_coreset_points, eps = self.eps_coreset)
            self.memory_bank = mem_concat[coreset_indices]            
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = AdamP(self.parameters(), lr=1e-6, betas=(0.9, 0.999), weight_decay=1e-2)
        scheduler = CosineAnnealingLR(optimizer, T_max=10)  
        return [optimizer], [scheduler]

    def validation_step(self, batch, batch_idx):
        x, y = batch
        w,delta = self(x)
        loss = relaxedContrastiveLoss(w, delta, self.margin)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        w,delta = self(x)
        loss = relaxedContrastiveLoss(w, delta, self.margin)
        self.log('test_loss', loss)
        return loss

In [ ]:
wandb.login(key = '4e6b8c4cffac1d8cf8ca1a28ff5ef5d77333d2de')
wandb.init(project='ReConPatch')
wandb_logger = WandbLogger(project='ReConPatch')
                                     
reconpatch = ReConPatch()
trainer = pl.Trainer(
    max_epochs=5, 
    logger=wandb_logger, 
    callbacks=[
        torchpl.callbacks.ModelCheckpoint(
            monitor='val_loss', 
            save_top_k=1, 
            mode='min',
            save_last=True
        ),
        torchpl.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=10,
            mode="min"),
      ], 
    devices=[0],
)
trainer.fit(model=reconpatch, datamodule=datamod)

#close the wandb session
wandb.finish()

In [ ]:

#reconpatch = ReConPatch.load_from_checkpoint("/kaggle/input/checkpoint/epoch26-step9801.ckpt")

In [ ]:
mem_bank = reconpatch.memory_bank
mem_bank = torch.tensor(mem_bank)

In [ ]:
#reconpatch = ReConPatch()

In [ ]:
# sample, target = next(iter(datamod.train_dataloader()))
# with torch.no_grad():
#     y = reconpatch(sample)
#     mem_bank = reconpatch.memory_bank

In [ ]:
print(mem_bank)

In [ ]:
# feature_dim=len(reconpatch.fmap)
# l=len(datamod.train_dataloader())

# #memory_bank=torch.zeros(l,6272,1536)#dataloader_size,feature_reshaped_size
# for i, values in enumerate(tqdm(datamod.train_dataloader())): 
#     img,label=values
#     y = reconpatch(img)
#     feature_map = reconpatch.fmap
# #     print(feature_map)
#     dimensions = (
#                 int(torch.Tensor([t.shape[-2] for t in feature_map]).max().item()),
#                 int(torch.Tensor([t.shape[-1] for t in feature_map]).max().item())
#     )
#     blur = nn.AvgPool2d(3, stride = 1)
#     resizer = nn.AdaptiveAvgPool2d(feature_map[0].shape[-2])
#     preprocess = lambda t : resizer(blur(t))
#     feature_stacks = torch.cat([preprocess(m) for m in feature_map], dim=1)
#     feature_stacks_reshaped = feature_stacks.reshape(feature_stacks.shape[1], -1).T
#     print(feature_stacks_reshaped.shape)
#     break
#     memory_bank[i]=feature_stacks_reshaped

In [ ]:
# memory_bank = np.concatenate(memory_bank, axis = 0)

In [ ]:
# frac_rate= 0.01
# eps_coreset = 0.05
# if frac_rate < 1:
#     num_coreset_points = int(frac_rate * len(memory_bank))
#     coreset_indices = obtainCoreset(mem_bank, l=num_coreset_points, eps=eps_coreset)
#     mem_bankory = mem_bankory[coreset_indices]


In [ ]:
def anomalyDetection(batch_images):
    # Check if batch_images is a batch or a single image
    if batch_images.dim() == 3:
        batch_images = batch_images.unsqueeze(0)
        
    batch_size = batch_images.size(0)
    scores = []

    for i in range(batch_size):
        image = batch_images[i]

        feature_map = reconpatch.fmap
        dimensions = (
            int(torch.Tensor([t.shape[-2] for t in feature_map]).max().item()),
            int(torch.Tensor([t.shape[-1] for t in feature_map]).max().item())
        )

        blur = nn.AvgPool2d(3, stride=1)
        resizer = nn.AdaptiveAvgPool2d(dimensions)
        preprocess = lambda t: resizer(blur(t))
        feature_stacks = torch.cat([preprocess(m) for m in feature_map], dim=1)
        feature_stacks_reshaped = feature_stacks.reshape(feature_stacks.shape[1], -1).T

        distances = torch.cdist(feature_stacks_reshaped, mem_bank, p=2)
        dist_val, dist_val_idxs = torch.min(distances, dim=1)
        s_idx = torch.argmax(dist_val)
        s_star = torch.max(dist_val)
        m_test_star = torch.unsqueeze(feature_stacks_reshaped[s_idx], dim=0)
        m_star = mem_bank[dist_val_idxs[s_idx]].unsqueeze(0)

        knn_dists = torch.cdist(m_star, mem_bank, p=2.0)
        _, nn_idxs = knn_dists.topk(k=3, largest=False)

        m_star_neighbourhood = mem_bank[nn_idxs[0, 1:]]
        w_denominator = torch.linalg.norm(m_test_star - m_star_neighbourhood, dim=1)
        norm = torch.sqrt(torch.tensor(feature_stacks_reshaped.shape[1]))
        w = 1 - (torch.exp(s_star / norm) / torch.sum(torch.exp(w_denominator / norm)))
        s = w * s_star

        scores.append(s)

    return torch.stack(scores)

In [ ]:

img_list=[]
labels_list=[]
for img,label in tqdm(datamod.test_dataloader()):
    #print(type(img[0][0][0][0].item()))
    labels_list.append(label)
   
    probs = anomalyDetection(img)
    probs=probs.to('cpu')
    img_list.append(probs)


    

In [ ]:
print(img_list[400])

In [ ]:
from torchmetrics import AUROC
print('img_list',len(img_list))
print('label_list_shape',len(labels_list))
#labels_list= torch.cat(labels_list,dim=0)
#img_list= torch.cat(img_list,dim=0)
auroc = AUROC(task="multiclass", num_classes=15)
auroc.update(img_list, labels_list)
auroc_value = auroc.compute()
print(auroc_value.item())